In [0]:
from pyspark.sql.functions import *

In [0]:
%run /Workspace/Users/aslammv2010@gmail.com/inventory_project_v1/wms/helper


In [0]:
fileLookup_df = (
    spark.read.table("inventory_project.metadata.filelookup")
         .filter((col("schema") == "silver") & (col("description").startswith("WMS")))
)
# fileLookup_df = fileLookup_df.collect()
display(fileLookup_df)

In [0]:
fileLookup_df = (
    spark.read.table("inventory_project.metadata.filelookup")
         .filter((col("schema") == "silver") & (col("description").startswith("WMS")))
)
fileLookup_df = fileLookup_df.collect()

audit_rows = []
for row in fileLookup_df:
    name = row['table_name']
    nbPath = row['notebookPath']
    status = 'success'
    start_time = datetime.now()
    try:
        print(f"Started processing {row['table_name']}")
        result = dbutils.notebook.run(nbPath, 600)
        print(result)
    except Exception as e:
        status = 'failed'
        error = str(e)
        print(f"Error processing {row['table_name']}: {e}")
        raise e
    finally:
        print(f"Finished processing {row['table_name']}")
        end_time = datetime.now()
        print(f"Notebook {name} took {end_time-start_time} seconds to run")
        updateFileLookup(name,status)
        audit_rows.append(Row(table_name = name, 
                              start_time = start_time, 
                              end_time = end_time, 
                              duration = str((end_time-start_time).total_seconds()), 
                              status = status,
                              error_message = error,
                              created_by = user,
                              created_date = datetime.now()))

In [0]:
# Add entry to audit table
if audit_rows:   # only if list is not empty
    audit_df = spark.createDataFrame(audit_rows).withColumn('created_by', lit('system'))
    audit_df.write \
        .format("delta") \
        .mode("append") \
        .option("mergeSchema", "true")\
        .saveAsTable("inventory_project.metadata.audit_log")